# 1. Exercise of saving model parameter etc.

In [ ]:
# import library
import tensorflow as tf

In [ ]:
# MNIST data download
# the below codes don't work
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_set("/tmp/data/", one_hot=True)

# alternative
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)


train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

## Define CNN model

In [ ]:
# define CNN model
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        # the first Convolution layer
        # apply 32 filters with 5x5 Kernel size
        self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
        self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)
        
        # the second Convolution layer
        # apply 64 filters with 5x5 Kernel size
        self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)
        
        #Fully connected Layer
        # Change 7x7 64 activation maps to 1024 features
        self.flatten_layer = tf.keras.layers.Flatten()
        self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')
        
        # Output Layer
        # Change 1024 features as 10 classes by one-hot encodding
        self.output_layer = tf.keras.layers.Dense(10, activation=None)
        
    def call(self,x):
        # reshape MNIST data as 3 dims
        x_image = tf.reshape(x, [-1,28,28,1])
        # 28x28x1 -> 28x28x32
        h_conv1 = self.conv_layer_1(x_image)
        # 28x28x32 -> 14x14x32
        h_pool1 = self.pool_layer_1(h_conv1)
        # 14x14x32 -> 14x14x64
        h_conv2 = self.conv_layer_2(h_pool1)
        # 14x14x64 -> 7x7x64
        h_pool2 = self.pool_layer_2(h_conv2)
        # 7x7x64(3136) -> 1024
        h_pool2_flat = self.flatten_layer(h_pool2)
        h_fc1 = self.fc_layer_1(h_pool2_flat)
        # 1024 -> 10
        logits = self.output_layer(h_fc1)
        y_pred = tf.nn.softmax(logits)

        return y_pred, logits

In [ ]:
# difine cross-entropy loss function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [ ]:
# define optimizer for optimization
optimizer = tf.optimizers.Adam(1e-4)

# function for optimization
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        y_pred, logits = model(x)
        loss = cross_entropy_loss(logits, y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
# function for printing model's accuracy
def compute_accuracy(y_pred, y):
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

In [ ]:
# declare model
CNN_model = CNN()

## Saving model

In [ ]:
# saving parameter using tf.train.CheckpointManager

SAVER_DIR = "./model"
ckpt = tf.train.Checkpoint(step=tf.Variable(0), model = CNN_model)
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=SAVER_DIR ,max_to_keep=5)
latest_ckpt = tf.train.latest_checkpoint(SAVER_DIR)

## Restore model

In [ ]:
# if there is a saved model and parameter then restore it
# and by using it print precision of test dataset and quit the program
if latest_ckpt:
    ckpt.restore(latest_ckpt)
    print("Accuracy (Restored): %f" % compute_accuracy(CNN_model(x_test)[0], y_test))
    exit()

Precision (Restored): 0.982800


In [ ]:
# optimize for 10000 step
while int(ckpt.step) < (10000 + 1):
    # load the 50 MNIST data at a time
    batch_x, batch_y = next(train_data_iter)
    
    # print accuracy of train dataset in each 100 step, then save parameter using tf.train.CheckpointManager
    if ckpt.step % 100 == 0:
        ckpt_manager.save(checkpoint_number=ckpt.step)
        train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
        print("Epoch: %d, Accuracy of training data: %f" % (ckpt.step, train_accuracy))
        
    # update parameter using optimizer
    train_step(CNN_model, batch_x, batch_y)
    ckpt.step.assign_add(1)

# if training is end, then print the accuracy
print("Accuracy (Restored): %f" % compute_accuracy(CNN_model(x_test)[0], y_test))

Epoch: 1700, Precision of training data: 1.000000
Epoch: 1800, Precision of training data: 0.960000
Epoch: 1900, Precision of training data: 0.980000
Epoch: 2000, Precision of training data: 0.940000
Epoch: 2100, Precision of training data: 0.960000
Epoch: 2200, Precision of training data: 0.980000


KeyboardInterrupt: 

# 2. Exercise of using TenserBoard

In [1]:
# TensorBoard 예제
# Convolutional Neural Networks(CNN)을 이용한 MNIST 분류기(Classifier) - Keras API를 이용한 구현

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

# tf.keras.Model을 이용해서 CNN 모델을 정의합니다.
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    # 첫번째 Convolution Layer
    # 5x5 Kernel Size를 가진 32개의 Filter를 적용합니다.
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    # 두번째 Convolutional Layer
    # 5x5 Kernel Size를 가진 64개의 Filter를 적용합니다.
    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    # Fully Connected Layer
    # 7x7 크기를 가진 64개의 activation map을 1024개의 특징들로 변환합니다.
    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    # Output Layer
    # 1024개의 특징들(feature)을 10개의 클래스-one-hot encoding으로 표현된 숫자 0~9-로 변환합니다.
    self.output_layer = tf.keras.layers.Dense(10, activation=None)

  def call(self, x):
    # MNIST 데이터를 3차원 형태로 reshape합니다. MNIST 데이터는 grayscale 이미지기 때문에 3번째차원(컬러채널)의 값은 1입니다.
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # 28x28x1 -> 28x28x32
    h_conv1 = self.conv_layer_1(x_image)
    # 28x28x32 -> 14x14x32
    h_pool1 = self.pool_layer_1(h_conv1)
    # 14x14x32 -> 14x14x64
    h_conv2 = self.conv_layer_2(h_pool1)
    # 14x14x64 -> 7x7x64
    h_pool2 = self.pool_layer_2(h_conv2)
    # 7x7x64(3136) -> 1024
    h_pool2_flat = self.flatten_layer(h_pool2)
    h_fc1 = self.fc_layer_1(h_pool2_flat)
    # 1024 -> 10
    logits = self.output_layer(h_fc1)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(1e-4)

11490434/11490434 [==============================] - 0s 0us/step


## Code for TensofBoard

In [9]:
# Set the directory of folder for summary info. and delcare FIlewriter
train_summary_writer = tf.summary.create_file_writer('./tensorboard_log/train')
test_summary_writer = tf.summary.create_file_writer('./tesnsorboard_log/test')

In [10]:
# function for optimization
def train_step(model, x,y):
    with tf.GradientTape() as tape:
        y_pred, logits = model(x)
        loss = cross_entropy_loss(logits, y)
    # save tf.summary.sclar, tf.summary.image tensor log at each step
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss, step=optimizer.iterations)
        x_image = tf.reshape(x, [-1,28,28,1])
        tf.summary.image('training image', x_image, max_outputs=10, step=optimizer.iterations)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
# function for printing accuracy
def compute_accuracy(y_pred, y, summary_writer):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with summary_writer.as_default():
        tf.summary.scalar('accuracy', accuracy, step=optimizer.iterations)
        
    return(accuracy)

In [11]:
# Convolutional Neural Networks(CNN) 모델을 선언합니다.
CNN_model = CNN()

In [ ]:
# optimizing for 10000 steps
for i in range(10000):
    # call 50 MNIST data
    batch_x, batch_y = next(train_data_iter)
    
    # print accuracy at each 100 steps
    if i % 100 == 0:
        train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y, train_summary_writer)
        print("Epoch: %d, Training Accuracy: %f" % (i, train_accuracy))
        
    # updating parameter using optimizer
    train_step(CNN_model, batch_x, batch_y)
    
#print Accuracy after fininshed trining
print("Accuracy: %f" % compute_accuracy(CNN_model(x_test)[0], y_test, test_summary_writer))